In [1]:
using Pkg

Pkg.activate("Agate")
Pkg.status()

  Activating project at `~/Documents/Projects/Agate.jl`


Project Agate v0.0.1
Status `~/Documents/Projects/Agate.jl/Project.toml`
  [a49af516] OceanBioME v0.11.1
  [9e8cae18] Oceananigans v0.91.13


In [2]:
using Agate

# The simplest case

The user can specify their own tracer functions, which need to be specified as a dictionary. The functions have to be written as Julia expression objects (they have to be defined inside `:(...)`. One can also optionally specify auxiliary fields as a vector (here one can list other parameters that the tracers depend on).

In [3]:
tracers = Dict(
    "N" => :(N + Z + D + P),
    "P" => :(0.5 * PAR),
    "D" => :(0.1 * D),
    "Z" => :(P-1)
)

aux_field_vars = [:PAR,]

1-element Vector{Symbol}:
 :PAR

Then one creates an Oceananigans BGC model by calling `construct_bgc_model`:

In [4]:
model = construct_bgc_model(tracers, auxiliary_fields=aux_field_vars)

Agate.Models.Dynamic.DynamicBGC()

Each tracer function can now be evaluated (note that there are 4 additional input parameters)

In [5]:
# inputs are: x,y,z,t,Z,P,N,D,PAR
model(Val(:N), 0, 0, 0, 0, 1, 1, 1, 1, 1)

4

# Using Agate functions

We can also make use of existing functions in the Agate library when specifying the tracer functions. To do that, the users also have to pass a dictionary of values for the constant terms in each function.

In [6]:
priors = Dict(
    :μ₀ => 0.6989,
    :kₙ => 2.3868,
    :lᵖⁿ => 0.066,
    :lᶻⁿ => 0.0102,
    :lᵖᵈ => 0.0101,
    :gₘₐₓ => 2.1522,
    :kₚ => 0.5573,
    :β => 0.9116,
    :lᶻᵈ => 0.3395,
    :rᵈⁿ => 0.1213,
    :α => 0.1953,
)

Dict{Symbol, Float64} with 11 entries:
  :kₙ   => 2.3868
  :α    => 0.1953
  :lᵖⁿ  => 0.066
  :rᵈⁿ  => 0.1213
  :lᶻᵈ  => 0.3395
  :μ₀   => 0.6989
  :lᶻⁿ  => 0.0102
  :lᵖᵈ  => 0.0101
  :β    => 0.9116
  :kₚ   => 0.5573
  :gₘₐₓ => 2.1522

In [7]:
tracers = Dict(
    "N" => :(phytoplankton_metabolic_loss(P, lᵖⁿ) 
    + zooplankton_metabolic_loss(Z, lᶻⁿ) 
    + remineralization(D, rᵈⁿ) 
    - phytoplankton_growth(N, PAR, P, μ₀, kₙ, α)),

    "D" => :(phytoplankton_mortality_loss(P, lᵖᵈ)
    + zooplankton_assimilation_loss(P, Z, β, gₘₐₓ, kₚ) 
    + zooplankton_mortality_loss(Z, lᶻᵈ) 
    - remineralization(D, rᵈⁿ) ),
    
    "P" => :(phytoplankton_growth(N, PAR, P, μ₀, kₙ, α) 
    - phytoplankton_grazing_loss(P, Z, gₘₐₓ, kₚ) 
    - phytoplankton_metabolic_loss(P, lᵖⁿ) 
    - phytoplankton_mortality_loss(P, lᵖᵈ)),
    
    "Z" => :(zooplankton_grazing_gain(P, Z, β, gₘₐₓ, kₚ) 
    - zooplankton_metabolic_loss(Z, lᶻⁿ)  
    - zooplankton_mortality_loss(Z, lᶻᵈ))
    )

aux_field_vars = [:PAR,]

1-element Vector{Symbol}:
 :PAR

In [8]:
model2 = construct_bgc_model(tracers, auxiliary_fields=aux_field_vars, priors=priors)

Agate.Models.Dynamic.DynamicBGC()

In [9]:
# the first 4 values should not matter - those are the x,y,z,t coordinates 
# This is followed by the tracers in this order: Z, P, N, D, PAR
model2(Val(:N), 0, 0, 0, 0, 1, 1, 1, 4, 1), model2(Val(:D), 0, 0, 0, 0, 3, 1, 1, 1, 1)

(0.5058625488175891, 3.3798033704115062)

### TEST

For comparison, we define a function for tracer N same as above and with all the same parameter values and show that we get the same result.

In [10]:
pwd()

"/Users/rjersakova/Documents/Projects/Agate.jl/notebooks"

In [11]:
# all the functions are defined here
include("../src/Models/Library.jl")

zooplankton_assimilation_loss (generic function with 1 method)

In [12]:
# just copy pasting expression from the top.... matching the arg order
μ₀=0.6989
kₙ=2.3868
lᵖⁿ=0.066
lᶻⁿ=0.0102
lᵖᵈ=0.0101
gₘₐₓ=2.1522
kₚ=0.5573
β=0.9116
lᶻᵈ=0.3395
rᵈⁿ=0.1213
α=0.1953

function N(Z, P, N, D, PAR)
    return phytoplankton_metabolic_loss(P, lᵖⁿ)  + zooplankton_metabolic_loss(Z, lᶻⁿ) + remineralization(D, rᵈⁿ)  - phytoplankton_growth(N, PAR, P, μ₀, kₙ, α)
end

function D(Z, P, N, D, PAR)
    phytoplankton_mortality_loss(P, lᵖᵈ) + zooplankton_assimilation_loss(P, Z, β, gₘₐₓ, kₚ)  + zooplankton_mortality_loss(Z, lᶻᵈ)   - remineralization(D, rᵈⁿ)
end

D (generic function with 1 method)

In [13]:
N(1, 1, 1, 4, 1), D(3, 1, 1, 1, 1)

(0.5058625488175891, 3.3798033704115062)

## Passing user defined methods

## Latex

Example of turning functions into latex

In [14]:
# return latex 
using Latexify
latexify(:(x + y)) # |> print

L"$x + y$"